# KED 재무 가공 노트북 (간단 비율 생성)

입력: `dataset/KED재무DATA.csv`
출력: `dataset/KED가공재무DATA.csv`

생성 지표 (4개, 연간 재무제표 기준 단순 계산):
- ROE = 당기순이익 / 자기자본
- 영업이익률 = 영업이익 / 매출액
- 부채비율 = 부채 / 자기자본
- 총자산회전율 = 매출액 / 자산

주의: 분모 0 또는 결측 시 결과는 NaN 처리합니다. 출력에는 식별자(사업자등록번호, 기준일자) + 위 4개 비율만 포함되며, 원본 수치 컬럼은 제외합니다.


In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

INPUT_PATH = Path('dataset/KED재무DATA.csv')
OUTPUT_PATH = Path('dataset/KED가공재무DATA.csv')

# 사용자 환경에 따라 열 이름이 다를 수 있습니다. 아래 기본 매핑을 확인/수정하세요.
# 예시 컬럼명 가정: 기준일자, 사업자등록번호, 매출액, 영업이익, 당기순이익, 자산, 자기자본, 부채
COLS = {
    'firm_id': '사업자등록번호',
    'date': '기준일자',
    'revenue': '매출액',
    'operating_income': '영업이익',
    'net_income': '당기순이익',
    'assets': '자산',
    'equity': '자기자본',
    'liabilities': '부채',
}

assert INPUT_PATH.exists(), f"입력 파일이 존재하지 않습니다: {INPUT_PATH}"

df = pd.read_csv(INPUT_PATH)

# 필수 컬럼 확인
for k, col in COLS.items():
    if col not in df.columns:
        raise ValueError(f"필수 컬럼 누락: {col} (키: {k})")

# 숫자형 변환
for key in ['revenue','operating_income','net_income','assets','equity','liabilities']:
    col = COLS[key]
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 안전한 나눗셈
def safe_div(n, d):
    return np.where((pd.notna(n)) & (pd.notna(d)) & (d != 0), n / d, np.nan)

# 단순 비율 계산 (연간 보고 기준)
res = df.copy()
res['ROE'] = safe_div(res[COLS['net_income']], res[COLS['equity']])
res['영업이익률'] = safe_div(res[COLS['operating_income']], res[COLS['revenue']])
res['부채비율'] = safe_div(res[COLS['liabilities']], res[COLS['equity']])
res['총자산회전율'] = safe_div(res[COLS['revenue']], res[COLS['assets']])

# 출력 컬럼: 기준일자, 사업자등록번호 + 4개 비율
keep_cols = [COLS['firm_id'], COLS['date'], 'ROE','영업이익률','부채비율','총자산회전율']
res = res[keep_cols]

# 저장
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
res.to_csv(OUTPUT_PATH, index=False, encoding='utf-8-sig')
print(f"✅ Saved: {OUTPUT_PATH} ({len(res)} rows, {len(keep_cols)} cols)")


✅ Saved: dataset\KED가공재무DATA.csv (723498 rows, 6 cols)
